本文主要内容来自：
- https://www.youtube.com/watch?v=5OYu0vxXEv8&list=PLJV_el3uVTsPy9oCRY30oBPNLCo89yu49&index=32

### 介绍
一般的DNN的input和output都只是一个vector。但在实际问题中，我们的output，可能是一个sequence，list或者是一个bounding box。和以前一样的，我们需要的是一个function f，在给定输入x的情况下，输出对应的值y。只不过，现在我们需要考虑的是，应该如何使用x，y来描述sequence，list和boounding box等。<br>
虽然structured learning听起来好像很困难，但是他有一个unifed framework。
![sl1](https://raw.githubusercontent.com/HuangYiran/readTech/master/fotos/sl1.png)
training的时候，我们要找一个F，他的input是X和Y，而output一个值R。这里的X和Y都是一个structured object，输出R代表X和Y的匹配程度。test的时候，我们就是要给出一个x，我们想看那个y，可以使得F最大。F和f的关系如图所示。<br>
插一句，就是structured learning和deep learning是很有关系的。GAN其实就是在实现structured learning的一个方法。<br>
### Domo
![sl2](https://raw.githubusercontent.com/HuangYiran/readTech/master/fotos/sl2.png)
任务是Object Detection。给定一个image，找出凉宫春日所在的Bounding Box。<br>
那么应该如何做呢？<br>
x在这里就是image，y就是bounding box。F(x,y)就是把bounding box框在image上，是有多正确。位置越对，F输出越高。<br>
testing的时候，就是要穷举所有的搭配，然后，输出分数最高的的搭配。<br>
### 实现的时候存在的问题
为了实现structured learning，还必须解决下面这三个问题
- Evaluation: F(x,y)应该长什么样
- 如何取最大值
- 如何training
![sl3](https://raw.githubusercontent.com/HuangYiran/readTech/master/fotos/sl3.png)
##### F(x, y)应该长什么样子
![sl5](https://raw.githubusercontent.com/HuangYiran/readTech/master/fotos/sl5.png)
F(x,y)有一个specific form，他是长这个样子的：<br>
首先，他必须是一个linear模型。也就是说，给定一组x，y对，首先，我们用一些特征来描述这对数据。这里我们用$/phi$来表示。<br>
然后，F(x,y)就被定义成，各个feature的加权就和。<br>
如图所示，最后我们的F(x,y)可以表示为：$F(x, y) = w\cdot \phi(x, y)$<br>
以上面Object detection的例子为例。我们把image和bounding box输出到方程$/phi$()中，希望得到一个vector，来描叙这个组合的不同属性，比如边框的大小、边框中黄色所占的比例等等。<br>
因为最后的F其实就是一个inner product，他没有办法得到很好的结果。所以为了最后能够得到很好的结果，我们就需要在这里得到很好的features。那么要如何得到很好的feature呢。在Object detection的例子中，我们会使用cnn来解决这个问题。如下图所示。
![sl6](https://raw.githubusercontent.com/HuangYiran/readTech/master/fotos/sl6.png)
##### training: separable case
我们假设问题2，已经得到解决，先来看一下，如何进行training。<br>
在这里我们的任务是，在给定一系列training set的情况下，我们要训练F，使得，在给出最佳配对的时候，F能够输出最大的值。具体来说就是：
![sl7](https://raw.githubusercontent.com/HuangYiran/readTech/master/fotos/sl7.png)
还是用Object detection来作为例子：
![sl8](https://raw.githubusercontent.com/HuangYiran/readTech/master/fotos/sl8.png)
这里原点第一张图片和不同的bounding box进行搭配，得到的特征，星星表示第二张图和不同的bounding box进行搭配，得到的特征。其中红色代表最佳搭配(问题2)。我们的目标是找到一个向量w，使得红色的点和w的inner product比其他相同形状的蓝色的点和w的inner product大。不同形状的点之间是互不干扰的。<br>
下面介绍一个用于找到对应w的算法:<br>
这个过程和perceptron learning是非常类似的。
![sl9](https://raw.githubusercontent.com/HuangYiran/readTech/master/fotos/sl9.png)
我们都知道，正对同一个的x，y的数量是惊人的。那么通过这个算法，我们要求的w，是否能够很快的收敛呢？？？<br>
我们的结论是：in separable case, to obtain a $\hat{w}$, you only have to update at most $(R/\sigma)^2$ times。这里$\sigma$是margin，R是同一个x，和所有不同y搭配，得到的$\phi$之间的最大距离。因此y的值域大小，并不影响update的次数。separable是指，存在vector，可以是得正确答案和错误答案被分开<br>

##### training2: unseparable case
下面来看一下unseparable case.<br>
在这种情况下，我们要定义一个cost function C。并用这个方程来定义一个w，他有多不好。使得w越好C越小。如下图：
![sl11](https://raw.githubusercontent.com/HuangYiran/readTech/master/fotos/sl11.png)
直观的说，就是第一名和正确的之间的距离。这里的$c^n$不可能是负的。<br>
现在我们有了C的表达式，下一步就是要找到最小化C的w。<br>
在这里我们可以是用(Stochastic)gradient Descent的方法。但是方程里有一个max存在，这样也可以使用gradient descent吗？事实上是可以的<br>
![sl12](https://raw.githubusercontent.com/HuangYiran/readTech/master/fotos/sl12.png)
如图中，不同区域代表在w固定的情况下，不同的y获得最大值的区域。所以在这些不同的区域，只要针对对应的y，做微分就行了。因此我们的算法如下：
![sl13](https://raw.githubusercontent.com/HuangYiran/readTech/master/fotos/sl13.png)

##### Inference????
现在来看一下第2个问题，获取使得F输出最大的y: $y = argmax_{y \in Y}w\cdot \phi(x,y)$<br>
还是以Object detection为例
首先，我们可以用穷举的方法.
另外，针对具体的问题，也存在特定的方法：
![sl10](https://raw.githubusercontent.com/HuangYiran/readTech/master/fotos/sl10.png)

### 和DNN的联系
![sl4](https://raw.githubusercontent.com/HuangYiran/readTech/master/fotos/sl4.png)